# Etude de marché de l'enseigne la poule qui chante

# Import des librairies

In [97]:
import pandas as pd
import numpy as np


from sklearn.preprocessing  import StandardScaler 
from sklearn.decomposition import PCA


import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import seaborn as sns
import scipy.stats as st

from sklearn import datasets
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix



# Chargement des données

In [98]:
dispo_alim_2017 = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
pop = pd.read_csv('Population_2000_2018.csv')

# Data Preparation

## Explorations des données

In [99]:
dispo_alim_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [100]:
dispo_alim_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176600 entries, 0 to 176599
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            176600 non-null  object 
 1   Domaine                 176600 non-null  object 
 2   Code zone               176600 non-null  int64  
 3   Zone                    176600 non-null  object 
 4   Code Élément            176600 non-null  int64  
 5   Élément                 176600 non-null  object 
 6   Code Produit            176600 non-null  int64  
 7   Produit                 176600 non-null  object 
 8   Code année              176600 non-null  int64  
 9   Année                   176600 non-null  int64  
 10  Unité                   176600 non-null  object 
 11  Valeur                  176600 non-null  float64
 12  Symbole                 176600 non-null  object 
 13  Description du Symbole  176600 non-null  object 
dtypes: float64(1), int64

# Transoposer le data set avec pivot_table

In [101]:
# transposer pour avoir plus de clarté sur les variables
df_dispo = pd.pivot_table (dispo_alim_2017 , index = ['Zone','Produit'], columns= ['Élément'], values = 'Valeur')

In [102]:
df_dispo=df_dispo.reset_index().rename_axis(None, axis=1)

In [103]:
df_dispo.tail()

,Zone,Produit,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
16042,Îles Salomon,Viande de Suides,0.0,NaN,NaN,44.0,4.49,4.24,1.38,3.0,NaN,0.0,3.0,NaN,2.0,0.0,NaN,NaN,0.0
16043,Îles Salomon,Viande de Volailles,0.0,NaN,NaN,18.0,4.45,1.31,1.51,3.0,0.0,6.0,3.0,NaN,0.0,0.0,NaN,NaN,3.0
16044,Îles Salomon,"Viande, Autre",0.0,NaN,NaN,0.0,0.01,0.00,0.00,0.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
16045,Îles Salomon,Vin,0.0,NaN,NaN,0.0,0.25,0.00,0.00,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
16046,Îles Salomon,"Épices, Autres",0.0,NaN,NaN,4.0,0.48,0.20,0.15,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0


Nous allons regarder s'il n'ya pas des valeurs manquantes ou des doublons dans nos data set

In [104]:
df_dispo.shape

(16047, 19)

### Vérification des doublons

In [105]:
df_dispo.duplicated().sum()

0

In [106]:
df_dispo.describe()

,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
count,5560.000000,4188.000000,5292.000000,14476.000000,14618.000000,14512.000000,14507.000000,15905.000000,12113.000000,15260.000000,14498.000000,5813.000000,10334.000000,12567.000000,2112.000000,3546.000000,11299.000000
mean,0.179496,361.943171,175.764928,33.962697,8.149033,1.045673,0.962651,694.599246,131.236110,96.930079,361.499448,105.257010,1090.312948,-3.043368,84.378314,735.305415,10.017258
std,3.589397,4045.919894,4979.484838,106.684337,24.304463,3.384062,3.248910,9717.089024,1376.616702,962.950046,5043.291136,1367.904219,12420.669023,52.383249,678.676108,8769.170271,381.004441
min,-44.000000,-2.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-828.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2912.000000,0.000000,0.000000,-10388.000000
25%,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,3.000000,0.610000,0.030000,0.030000,6.000000,0.000000,1.000000,4.000000,2.000000,15.000000,0.000000,2.000000,2.000000,0.000000
75%,0.000000,31.000000,4.000000,20.000000,4.647500,0.370000,0.400000,75.000000,10.000000,18.000000,49.000000,19.000000,154.000000,0.000000,18.000000,42.000000,0.000000
max,166.000000,187553.000000,319603.000000,1785.000000,457.700000,60.100000,48.680000,758548.000000,68156.000000,95538.000000,473967.000000,83476.000000,758548.000000,1870.000000,22501.000000,345510.000000,25089.000000


In [107]:
df_dispo.isnull().sum()

Zone                                                                 0
Produit                                                              0
Alimentation pour touristes                                      10487
Aliments pour animaux                                            11859
Autres utilisations (non alimentaire)                            10755
Disponibilité alimentaire (Kcal/personne/jour)                    1571
Disponibilité alimentaire en quantité (kg/personne/an)            1429
Disponibilité de matière grasse en quantité (g/personne/jour)     1535
Disponibilité de protéines en quantité (g/personne/jour)          1540
Disponibilité intérieure                                           142
Exportations - Quantité                                           3934
Importations - Quantité                                            787
Nourriture                                                        1549
Pertes                                                           10234
Produc

## Nettoyage

In [108]:
#filtrer sur les variables d'analyse
df_dispo=df_dispo.loc[df_dispo['Produit']=='Viande de Volailles',['Zone','Produit','Disponibilité alimentaire (Kcal/personne/jour)','Exportations - Quantité','Importations - Quantité','Production']]

In [109]:
df_dispo

,Zone,Produit,Disponibilité alimentaire (Kcal/personne/jour),Exportations - Quantité,Importations - Quantité,Production
78,Afghanistan,Viande de Volailles,5.0,NaN,29.0,28.0
173,Afrique du Sud,Viande de Volailles,143.0,63.0,514.0,1667.0
265,Albanie,Viande de Volailles,85.0,0.0,38.0,13.0
357,Algérie,Viande de Volailles,22.0,0.0,2.0,275.0
452,Allemagne,Viande de Volailles,71.0,646.0,842.0,1514.0
...,...,...,...,...,...,...
15668,Émirats arabes unis,Viande de Volailles,147.0,94.0,433.0,48.0
15765,Équateur,Viande de Volailles,83.0,0.0,0.0,340.0
15860,États-Unis d'Amérique,Viande de Volailles,219.0,3692.0,123.0,21914.0
15954,Éthiopie,Viande de Volailles,0.0,NaN,1.0,14.0


### Traitement des valeurs manquantes

Nous constatons que notre variable production contienne 8 valeurs nulls et 4 valeurs manquantes En revanche on en a 37 valeurs manquantes 
de notre variable exportations donc notre data set contient des VM sur les differentes variables qu'on remplacera par 0  lors de jointures

In [110]:
# renommer la colonne exportatio-quantité et import quantité
df_dispo.rename (columns={'Exportations - Quantité' : 'Exportations'}, inplace = True)
df_dispo.rename (columns={'Importations - Quantité' : 'Importations'}, inplace = True)


In [111]:
df_dispo.head()

,Zone,Produit,Disponibilité alimentaire (Kcal/personne/jour),Exportations,Importations,Production
78,Afghanistan,Viande de Volailles,5.0,NaN,29.0,28.0
173,Afrique du Sud,Viande de Volailles,143.0,63.0,514.0,1667.0
265,Albanie,Viande de Volailles,85.0,0.0,38.0,13.0
357,Algérie,Viande de Volailles,22.0,0.0,2.0,275.0
452,Allemagne,Viande de Volailles,71.0,646.0,842.0,1514.0


In [112]:
# exploration du data set population
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

In [113]:
#pivoter le data set population 
df_pop = pd.pivot_table (pop , index = ['Zone','Produit'], columns= ['Année'], values = 'Valeur')
df_pop=df_pop.reset_index().rename_axis(None, axis=1)
df_pop.head()

,Zone,Produit,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,Population-Estimations,20779.953,21606.988,22600.770,23680.871,24726.684,25654.277,26433.049,27100.536,...,28394.813,29185.507,30117.413,31161.376,32269.589,33370.794,34413.603,35383.032,36296.113,37171.921
1,Afrique du Sud,Population-Estimations,44967.708,45571.274,46150.913,46719.196,47291.610,47880.601,48489.459,49119.759,...,50477.011,51216.964,52003.755,52832.658,53687.121,54544.186,55386.367,56207.646,57009.756,57792.518
2,Albanie,Population-Estimations,3129.243,3129.704,3126.187,3118.023,3104.892,3086.810,3063.021,3033.998,...,2973.048,2948.023,2928.592,2914.096,2903.790,2896.305,2890.513,2886.438,2884.169,2882.740
3,Algérie,Population-Estimations,31042.235,31451.514,31855.109,32264.157,32692.163,33149.724,33641.002,34166.972,...,35333.881,35977.455,36661.445,37383.895,38140.133,38923.692,39728.025,40551.392,41389.189,42228.408
4,Allemagne,Population-Estimations,81400.882,81453.885,81535.122,81614.380,81646.474,81602.741,81472.226,81277.830,...,80899.961,80827.002,80855.632,80972.628,81174.367,81450.378,81787.411,82193.768,82658.409,83124.418


In [114]:
df_pop = df_pop[['Zone',2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]]

In [115]:
df_pop

,Zone,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,27722.276,28394.813,29185.507,30117.413,31161.376,32269.589,33370.794,34413.603,35383.032,36296.113,37171.921
1,Afrique du Sud,49779.471,50477.011,51216.964,52003.755,52832.658,53687.121,54544.186,55386.367,56207.646,57009.756,57792.518
2,Albanie,3002.678,2973.048,2948.023,2928.592,2914.096,2903.790,2896.305,2890.513,2886.438,2884.169,2882.740
3,Algérie,34730.608,35333.881,35977.455,36661.445,37383.895,38140.133,38923.692,39728.025,40551.392,41389.189,42228.408
4,Allemagne,81065.752,80899.961,80827.002,80855.632,80972.628,81174.367,81450.378,81787.411,82193.768,82658.409,83124.418
...,...,...,...,...,...,...,...,...,...,...,...,...
233,Îles Salomon,503.360,515.181,527.861,541.521,556.063,571.336,587.079,603.118,619.438,636.039,652.857
234,Îles Turques-et-Caïques,31.202,31.934,32.660,33.377,34.066,34.731,35.369,35.981,36.559,37.115,37.665
235,Îles Vierges américaines,106.756,106.412,106.087,105.793,105.534,105.303,105.114,104.958,104.835,104.751,104.680
236,Îles Vierges britanniques,26.097,27.039,27.794,28.319,28.650,28.847,28.989,29.152,29.355,29.577,29.802


### Jointure des data sets

In [116]:

# remplacer les valeurs manquantes par 0 et jointure des data set pop et dispo alim

df_dispo_pop = pd.merge(df_dispo.fillna(0), df_pop, on ='Zone', how = 'inner')

In [117]:
# creation de dataset pour la realisation del'acp

df_acp = df_dispo_pop[[  'Zone', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Exportations','Importations', 
                       'Production', 2008,2018]]
df_acp

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Exportations,Importations,Production,2008,2018
0,Afghanistan,5.0,0.0,29.0,28.0,27722.276,37171.921
1,Afrique du Sud,143.0,63.0,514.0,1667.0,49779.471,57792.518
2,Albanie,85.0,0.0,38.0,13.0,3002.678,2882.740
3,Algérie,22.0,0.0,2.0,275.0,34730.608,42228.408
4,Allemagne,71.0,646.0,842.0,1514.0,81065.752,83124.418
...,...,...,...,...,...,...,...
167,Émirats arabes unis,147.0,94.0,433.0,48.0,7089.487,9630.959
168,Équateur,83.0,0.0,0.0,340.0,14535.739,17084.358
169,États-Unis d'Amérique,219.0,3692.0,123.0,21914.0,303486.012,327096.265
170,Éthiopie,0.0,0.0,1.0,14.0,82916.235,109224.414


In [118]:
# exploration des nom des varibales qui constitue mon data set d'analyse
df_acp.columns

Index([                                          'Zone',
       'Disponibilité alimentaire (Kcal/personne/jour)',
                                         'Exportations',
                                         'Importations',
                                           'Production',
                                                   2008,
                                                   2018],
      dtype='object')

### Calcul taux de dependance d'importations et le t'aux d'autosiffisance alimentaire

In [119]:

#le TDI  permet de savoir quelle part les approvisionnements  
# intérieurs disponibles provient des importations et quelle part provient de la production du pays lui-même. 

df_acp['TDI'] = df_dispo_pop ['Importations'] *100 / (df_acp['Production']+df_acp['Importations'] - df_acp['Exportations'])


# Le TAS est un indice permettant de mesurer l'importance de la production alimentaire d'un pays par rapport
#à sa consommation intérieure.

df_acp ['TAS'] = df_acp['Production'] *100 / (df_acp['Production']+ df_acp['Importations'] - df_acp['Exportations'])


C:\Users\minaw\AppData\Local\Temp\ipykernel_3672\2069433930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_acp['TDI'] = df_dispo_pop ['Importations'] *100 / (df_acp['Production']+df_acp['Importations'] - df_acp['Exportations'])
C:\Users\minaw\AppData\Local\Temp\ipykernel_3672\2069433930.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_acp ['TAS'] = df_acp['Production'] *100 / (df_acp['Production']+ df_acp['Importations'] - df_acp['Exportations'])


In [120]:
#supprimer les valeurs manquantes

df_acp = df_acp.dropna()

# calculer l'evoluition de la population sur les 10 dernieres années

df_acp['evol_pop'] = (df_acp[2018]-df_acp[2008])/(df_acp[2008])

#suppression des colonnes après calcul
df_acp= df_acp.drop(columns = [2008,2018])
df_acp

C:\Users\minaw\AppData\Local\Temp\ipykernel_3672\831375907.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_acp['evol_pop'] = (df_acp[2018]-df_acp[2008])/(df_acp[2008])


,Zone,Disponibilité alimentaire (Kcal/personne/jour),Exportations,Importations,Production,TDI,TAS,evol_pop
0,Afghanistan,5.0,0.0,29.0,28.0,50.877193,49.122807,0.340868
1,Afrique du Sud,143.0,63.0,514.0,1667.0,24.268178,78.706327,0.160971
2,Albanie,85.0,0.0,38.0,13.0,74.509804,25.490196,-0.039944
3,Algérie,22.0,0.0,2.0,275.0,0.722022,99.277978,0.215885
4,Allemagne,71.0,646.0,842.0,1514.0,49.239766,88.538012,0.025395
...,...,...,...,...,...,...,...,...
167,Émirats arabes unis,147.0,94.0,433.0,48.0,111.886305,12.403101,0.358485
168,Équateur,83.0,0.0,0.0,340.0,0.000000,100.000000,0.175335
169,États-Unis d'Amérique,219.0,3692.0,123.0,21914.0,0.670482,119.454892,0.077797
170,Éthiopie,0.0,0.0,1.0,14.0,6.666667,93.333333,0.317286


In [121]:
df_acp=df_acp.reset_index(drop=True)

In [122]:
#supprimer les outliers

df_acp.drop(df_acp.loc[df_acp['Zone']=='Brésil'].index, inplace=True)
df_acp.drop(df_acp.loc[df_acp['Zone']=="États-Unis d'Amérique"].index, inplace=True)
df_acp.drop(df_acp.loc[df_acp['Zone']=='Chine - RAS de Hong-Kong'].index, inplace=True)
# on supprime la France car c'est un marché local
df_acp.drop(df_acp.loc[df_acp['Zone']=='France'].index, inplace=True)


#un gros producteur avec un TAS élevé donc un outlier qui peut biaiser mon clustering

df_acp.drop(df_acp.loc[df_acp['Zone']=='Inde'].index, inplace=True)

#supprimer les outliers Chine c'est un gros producteur de poulet avec un TDI faible 
df_acp.drop(df_acp.loc[df_acp['Zone']=='Chine, continentale'].index, inplace=True)

# USA et Brésil, produit exporte beaucoup avec dispo très élevé, donc c'est pas interessant de le garder dans mon echantion d'analyse
# Chine, Hong-kong c'est une île isolé qui importe pas.

In [123]:
df_acp.to_csv('df_acp.csv', index=False)